In [1]:
import numpy as np
import pandas as pd

from PIL import Image
from sklearn.svm import SVC

Upload date

In [2]:
train_data = pd.read_csv("./data/train.txt", sep=',')

validation_data = pd.read_csv("./data/validation.txt", sep=',')

test_data = pd.read_csv("./data/test.txt", sep=',')

Functie pentru citire o imagine dupa filename si return pixels

In [3]:
def return_pixel_rgb_values(img_name, path = "./data/train_validation/") :
    return np.array(list(Image.open(path + img_name).getdata())).reshape((16, 16, 3))

Incarcare pixeli

In [4]:
train_images = np.array([return_pixel_rgb_values(x) for x in train_data.id])

validation_images = np.array([return_pixel_rgb_values(x) for x in validation_data.id]).reshape((1173, 768))/255

test_images = np.array([return_pixel_rgb_values(x, "./data/test/") for x in test_data.id]).reshape((2819, 768)) / 255


"""Aici o sa augmentam datele si vom roti imaginile cu 90 grade"""

train_rotate_90 = np.rot90(train_images, k=1, axes=(0, 1)).reshape((8000,768)) / 255

train_images = train_images.reshape((8000, 768)) / 255

Calculam media imaginilor la fiecare categorie

In [5]:
train_mean_image = np.mean(train_images, axis=0)

train_90_mean = np.mean(train_rotate_90, axis=0)

validation_mean_image = np.mean(validation_images, axis=0)

test_mean_image = np.mean(test_images, axis=0)

Standardizam datele

In [6]:
train_images_standardized = train_images - train_mean_image / np.std(train_images)

train_90_standardized = train_rotate_90 - train_90_mean / np.std(train_rotate_90)

validation_images_standardized = validation_images - validation_mean_image/np.std(validation_images)

test_images_standardized = test_images - test_mean_image / np.std(test_images)

In [7]:
images_perturbed = []
for img in train_images_standardized :
    images_perturbed.append(img + np.random.normal(loc=0.0,scale=0.01, size=(768,)))
images_perturbed = np.array(images_perturbed)

In [8]:
a = np.concatenate((train_images_standardized, train_90_standardized, images_perturbed), axis=0)
a_labels = np.concatenate((np.array(train_data.label), np.array(train_data.label), np.array(train_data.label)), axis=0)

In [9]:
a_labels.shape

(24000,)

In continuare avem SVM ul

Aici am def o functie care ne creaza si antreneaza modelul nostru

In [10]:
def create_model(train_arrays, train_labels, c_parameter = 1.0, kernel = "linear") :

    model_SVM = SVC(C=c_parameter, kernel=kernel)

    model_SVM.fit(train_arrays, train_labels)

    return model_SVM

In [8]:
model_svm = create_model(a, a_labels, c_parameter=1.0)

In [9]:
(model_svm.predict(validation_images_standardized) == np.array(validation_data.label)).mean()

0.46462063086104005

In [ ]:
for c in [0.1,0.5,1.0,1.8,3.0] :
    model_svm = create_model(a, a_labels, c_parameter=c)
    print(f"Valoarea pt \tc = \t{c} \teste = \t{(model_svm.predict(validation_images_standardized) == np.array(validation_data.label)).mean()}")

In [ ]:
for c in [1.0] :
    model_svm = create_model(a, a_labels, c_parameter=c, kernel = "rbf")
    print(f"Valoarea pt \tc = \t{c} \teste = \t{(model_svm.predict(validation_images_standardized) == np.array(validation_data.label)).mean()}")

In [ ]:
for c in [0.6, 1.1, 2.2,3.3] :
    model_svm = create_model(a, a_labels, c_parameter=c, kernel = "rbf")
    labels = model_svm.predict(validation_images_standardized)
    print(f"Valoarea pt \tc = \t{c} \teste = \t{(labels == np.array(validation_data.label)).mean()}")

Valoarea pt 	c = 	0.6 	este = 	0.5311167945439045
Valoarea pt 	c = 	1.1 	este = 	0.5498721227621484
Valoarea pt 	c = 	2.2 	este = 	0.5481670929241261


In [11]:
b = np.concatenate((a, validation_images_standardized), axis=0)
b_labels = np.concatenate((a_labels, np.array(validation_data.label)), axis=0)

for c in [1.1] :
    model_svm = create_model(b, b_labels, c_parameter=c, kernel = "rbf")
    labels = model_svm.predict(test_images_standardized)
    # print(f"Valoarea pt \tc = \t{c} \teste = \t{(labels == np.array(validation_data.label)).mean()}")

In [15]:
labels

array([0, 6, 2, ..., 3, 1, 0], dtype=int64)

In [16]:
f = open("svm_third_try_c_is_1p1.txt","w")
f.write("id,label\n")
i = 0
for nume_img in test_data.id :
    f.write(f"{nume_img},{labels[i]}\n")
    i += 1

acum facem fisieru out si cu datele de validare

In [14]:
b = np.concatenate((a, validation_images_standardized), axis=0)
b_labels = np.concatenate((a_labels, np.array(validation_data.label)), axis=0)

for c in [3.3] :
    model_svm = create_model(b, b_labels, c_parameter=c, kernel = "rbf")
    labels2 = model_svm.predict(test_images_standardized)
    # print(f"Valoarea pt \tc = \t{c} \teste = \t{(labels == np.array(validation_data.label)).mean()}")

In [16]:
f = open("svm_second_try_c_is_3p3.txt","w")
f.write("id,label\n")
i = 0
for nume_img in test_data.id :
    f.write(f"{nume_img},{labels2[i]}\n")
    i += 1